In [138]:
import numpy as np
import pandas as pdcoding
import gzip

In [139]:
#SOMToolbox Parser
from SOMToolBox_Parse import SOMToolBox_Parse
idata = SOMToolBox_Parse("datasets\\10clusters\\10clusters.vec").read_weight_file()
weights = SOMToolBox_Parse("datasets\\10clusters\\10clusters.wgt").read_weight_file()
#units holds all information about what input vectors are mapped to which unit
units = SOMToolBox_Parse("datasets\\10clusters\\10clusters.unit").read_unit_file()
tv = SOMToolBox_Parse("datasets\\10clusters\\10clusters.tv").read_tv_file()
max_shown_values = 4


In [140]:
from statistics import stdev, mean
from math import pow

def LabelSom(_m, _n, _weights, _idata, _units, _num_of_attr, std_threshold = None, mean_threshold = None, unitBased = True):
    data = {}
    arr = np.zeros( weights['ydim'] *  weights['xdim'], dtype=int).reshape(weights['ydim'],weights['xdim'])
    for (x,y), unit in _units.items():
        statistics = []
        show_attr = []
        for attr in range(_num_of_attr):
            # Get all values of the the attr attribute for the current unit
            attr_vals = list(map(lambda x: _idata[x - 1][attr] ,unit["mapped_vecs"]))
            if len(attr_vals) > 0:
                statistics.append((mean(attr_vals), abs(stdev(attr_vals))))
                if abs(stdev(attr_vals)) < std_threshold:
                    show_attr.append(attr+1)
                    #Encode if the attribute is under the threshold by using power of 2 
                    # so we can decode it with binary and have a 1 on the attr-th place (reading from left to right)
                    arr[x,y] += pow(2, _num_of_attr - attr) 
        data[(x,y)] = show_attr
    
    return arr.reshape(weights['ydim'],weights['xdim']) #data


def get_label_string(tv, num, max_shown_values):
    labels = []
    for i, aktiv in enumerate("{0:b}".format(num)):
        if aktiv == "1":
            labels.append(tv[int(i+1)])
            if len(labels) > max_shown_values - 1:
                break
    return '\n'.join(labels)
             

In [148]:
import panel as pn
import holoviews as hv
from holoviews import opts
from bokeh.plotting import show

labelSom = LabelSom(weights['ydim'], weights['xdim'], weights['arr'], idata['arr'], units, idata['vec_dim'], 0.1)
h = hv.Image(labelSom).opts(xaxis=None, yaxis=None,color_levels=1)
hvdim_value = hv.Dimension("value", value_format=lambda x: get_label_string(tv, x, max_shown_values))
im = h * hv.Labels(h, vdims=hvdim_value).opts(
    padding=0, text_color="black", text_alpha=1.0, text_font_size="5pt"
).opts(xaxis=False, yaxis=False) 

show(hv.render(im))